In [6]:
import pandas as pd
import os
import gzip
import pickle
import openai

In [7]:
categories = [
    "groupbuys",
    "hardware",
    "miners",
    "mining",
    "mining_support",
    "pools",
]

df = pd.DataFrame()

#load every csv in the folder and append them
for cat in categories:
    with gzip.open('cleaned-data/'+cat+'.pkl.gz', 'rb') as f:
        df_cat = pickle.load(f)
        df_cat['category'] = cat
        df = pd.concat([df, df_cat], ignore_index=True)
    

In [8]:
df.sample(10)

,topic,post,date,dates,category
2337,[OPEN] #KN-R001 3TH/s KnCMiner Neptune - FREE ...,-Reserved-<sep>Fantastic product and presentat...,2013-12-05 20:43:17,2013-11-27 11:55:17<sep>2013-11-27 20:40:33<se...,groupbuys
4115,[SALE] Discounted Icarus FPGA [UPGRADABLE to A...,Trade-ins only apply to the future full price ...,2012-10-14 17:08:56,2012-10-14 16:03:49<sep>2012-10-14 17:08:56,hardware
1422,[PAYPAL- IN STOCK / 107 Units left] $32.95!~OR...,Almost under 100 units left! All orders ship T...,2013-09-02 02:43:07,2013-09-02 02:43:07,groupbuys
15914,[12 BTC Bounty]charitycoin - 0% fees + benefit...,Updated to 0% fees for the first 6 months for ...,2011-07-18 11:45:00,2011-07-18 11:45:00,pools
15815,[ 2.3 GH/s ] Mainframe.nl Mining Cooperative -...,Signed also up.So lets get that first block.<s...,2011-07-07 23:16:56,2011-07-07 22:51:39<sep>2011-07-07 23:16:56,pools
5027,MinersCube ... Anybody place an order ?,"Has anyone placed an order with MinerCube, now...",2014-06-08 04:38:30,2014-06-08 02:49:24<sep>2014-06-08 04:25:42<se...,hardware
7364,Mhash Charting Software,Does anybody know if this type of app exists y...,2011-07-02 13:14:29,2011-06-29 15:25:41<sep>2011-06-30 09:44:39<se...,miners
2937,### GRIDSEED Group-Buy Shares ### 26 Million S...,Bought 10 million shares. Make me happy please...,2014-02-18 12:45:24,2014-02-18 05:46:46<sep>2014-02-18 05:53:55<se...,groupbuys
13532,test psu independently from miner,"is it easy, or even possible without buying so...",2017-03-23 20:15:47,2017-03-22 23:32:16<sep>2017-03-22 23:44:05<se...,mining_support
4376,HashTech Avalon Clone Hashing Modules 22GH/s -...,Pre-order put down. Looking forward to it!<sep...,2013-06-16 21:06:07,2013-06-15 18:09:40<sep>2013-06-15 18:11:16<se...,hardware


In [69]:
def get_openai_response(prompt, tokens = 2000, model="gpt-3.5-turbo-instruct"):
    if model.__contains__("instruct"):
        response = openai.Completion.create(
            model=model,
            # messages=[
            #     {
            #         "role": "system",
            #         "content": "Answer the prompt with a single word"
            #     },
            #     {
            #         "role": "user",
            #         "content": prompt
            #     }
            #     ],
            prompt=prompt,
            temperature=0,
            max_tokens=tokens,
            top_p=1,
            logit_bias = {
                "198": -100, # new lines
                "4521": -100, # ---
                },
            logprobs= 15,
        )
        text = response.choices[0].text
        print(response.choices[0].logprobs)

        # print("returning ai text:", text)
        return text
    else:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant"
                },
                {
                    "role": "user",
                    "content": prompt
                }
                ],
            temperature=0,
            max_tokens=tokens,
            top_p=1,
        )
        text = response.choices[0].message.content

        # print("returning ai text:", text)
        return text

In [70]:
prompt = """
You are an expert at analysing forums.

The following is a randomly chosen thread from a bitcoin forum.

I need to know if the thread contains a name of mining hardware. 
Note: 
-Mining pools are not hardware.
-Companies that sell mining hardware, such as Bitmain, Microbt and Bitfury are not hardware.

Example hardware names are ARM Cortex A9, X6500, AvalonMiner 1, Jupiter, RockerBox, BE300, PickAxe, Antminer S9 (there are many more)
Sometimes NVIDIA and AMD GPUs are mentioned by their number only. For example, "I have an issue with a 5970". Those are also valid hardware names.

If the thread contains a name of mining hardware, say its name. Otherwise, say "No".

The thread:
""".strip()

for (id,row) in df.sample(1).iterrows():
    if(len(row["post"]) < 100):
        continue
    continue

    thread = ""
    thread += "category: " + row["category"] + "\n"
    thread += "topic: " + row["topic"] + "\n"
    thread += "date: " + row["dates"].split("<sep>")[-1][:7] + "\n"
    thread += "posts:\n"
    for (post, date) in zip(row["post"].split("<sep>"), row["dates"].split("<sep>")):
        if len(post) > 1000:
            thread += post[:1000] + "<rest of post truncated>\n"
            thread += "---\n"
        elif len(post) < 5:
            pass
        else:
            thread += post + "\n"
            thread += "---\n"
    thread = thread.strip()[:-3]
    if len(thread) > 6000:
        thread = thread[:6000] + "<rest of thread truncated>\n"
    
    print(thread)

    response = get_openai_response(prompt + "\n" + thread, tokens = 3)
    print(response)

In [76]:
prompt = """
You are an expert at analysing forums.

The following is a randomly chosen thread from a bitcoin forum.

I need to know if the thread contains a name of mining hardware. 
Note: 
-Mining pools are not hardware.
-Companies that sell mining hardware, such as Bitmain, Microbt and Bitfury are not hardware.

Example hardware names are ARM Cortex A9, X6500, AvalonMiner 1, Jupiter, RockerBox, BE300, PickAxe, Antminer S9 (there are many more)
Sometimes NVIDIA and AMD GPUs are mentioned by their number only. For example, "I have an issue with a 5850". Those are also valid hardware names.

If the thread contains a name of mining hardware, say its name. Otherwise, say "No".

The thread:
""".strip()

thread = """
category: mining_support
topic: Any help? - 5970 not hitting ~700 mhash/s
date: 2012-02

original post:
Just got my 5970 in the mail and got it set up and it's getting me only ~550 mhash/s using GUIMiner.

reply 1:
Look for SDK 2.1.
""".strip()

# thread = """
# category: pools
# topic: Ghash.io good or bad
# date: 2014-01
# original post:
# do you think that ghash.iois good or bad pool??//

# reply 1:
# The pool is not bad but be prepared to sometime lose and other times to win. As a mining pool not bad as very good total hash.

# reply 2:
# Good if we use great hardware for mining :d
# Nothing wrong with the pool itself, but their closing in on 51% is VERY BAD.
# """.strip()

actual_prompt = prompt + "\n" + thread + "\n\n" + "As an expert at analysing forum posts, my answer is:"


response = get_openai_response(actual_prompt, tokens = 10)
# response = get_openai_response(prompt + "\n" + thread, tokens = 3, model="gpt-4")
print(response)

{
  "tokens": [
    " Yes",
    ",",
    " the",
    " thread",
    " contains",
    " a",
    " name",
    " of",
    " mining",
    " hardware"
  ],
  "token_logprobs": [
    -1.4301753,
    -0.039598607,
    -0.043495722,
    -0.0657889,
    -0.38210675,
    -0.15292111,
    -0.07292792,
    -6.468596e-05,
    -0.000599397,
    -3.5120327e-05
  ],
  "top_logprobs": [
    {
      " Yes": -1.4301753,
      " No": -1.8520505,
      " AMD": -2.0239253,
      "\n\n": -2.1489255,
      " \"": -2.5864255
    },
    {
      ",": -0.039598607,
      ".": -3.4145985,
      " -": -5.8052235,
      ".\n\n": -7.336473,
      ".\n": -7.664598
    },
    {
      " the": -0.043495722,
      " this": -4.0278707,
      " ": -4.543495,
      " it": -5.1997457,
      " AMD": -5.4809957
    },
    {
      " thread": -0.0657889,
      " post": -3.9095387,
      " ": -4.2689137,
      " original": -4.4095387,
      " hardware": -4.4095387
    },
    {
      " contains": -0.38210675,
      " mentions": -1.